In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import get_loader
from data_loader import get_encoder_loader
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [17]:
batch_size = 128
num_epochs = 50
extract_size = 2048
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
encoder = Encoder()
encoder = encoder.to(device)

# data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size)
# data_loader.dataset.save_to("./bottle_neck/resnet152_train.h")
data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size, file='./bottle_neck/resnet152_train.h')

### init model

In [4]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = MultiClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(classify_model.parameters())

### time to train model

In [18]:
for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f' % (epoch, num_epochs, i_step, total_step, loss.item())
        
        print('\r' + stats, end='')
        sys.stdout.flush()
    if epoch%50 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_multi_model_%d.pkl' % epoch))
torch.save(classify_model.state_dict(), os.path.join('./models', 'class_multi_model_last.pkl'))

Epoch [600/400], Step [255/255], Loss: 0.2567

### How good is the model?

In [19]:
# valid_data_loader = get_encoder_loader(transform_train, encoder, device, mode='test', batch_size=batch_size)
# valid_data_loader.dataset.save_to("./bottle_neck/resnet152_valid.h")
valid_data_loader = get_encoder_loader(transform_train, encoder, device, mode='test', batch_size=batch_size, file="./bottle_neck/resnet152_valid.h")

In [61]:
predict = []
for embed, _ in valid_data_loader.dataset:
    p = classify_model(embed).argmax().item()
    predict.append(p)

In [62]:
df_refer = valid_data_loader.dataset.refer
df_refer["predict"] = predict

In [63]:
df_refer['correct'] = df_refer.predict == df_refer.disease_class

In [64]:
accuracy = (df_refer.correct == True).sum()/len(df_refer)
print('The final accuracy is %.2f%%.' % (accuracy*100))

The final accuracy is 63.51%.
